## Tempe PNML Data Time Series Analysis
#### Drew Gobbi 12/1/2019

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
import geopandas as gpd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from time import strptime
import os 
import patsy 
from shapely.wkt import loads

In [4]:
two = pd.read_csv('tp_02_waterdata_weekly.csv')
four = pd.read_csv('tp_04_waterdata_weekly.csv')
five = pd.read_csv('tp_05_waterdata_weekly.csv')

In [5]:
five.shape

(784, 12)

In [8]:
for i in range(len(two)):
    print(two.Sample_Date[i])

2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-05-14 03:00:00
2018-05-15 03:00:00
2018-05-16 03:00:00
2018-05-17 03:00:00
2018-05-18 03:00:00
2018-05-19 03:00:00
2018-05-20 03:00:00
2018-07-23 03:00:00


In [ ]:
two.dtypes

In [ ]:
two.Chemical.unique()

In [ ]:
#focus just on heroin for now
two = two[two['Chemical'] == 'Heroin']
four = four[four['Chemical'] == 'Heroin']
five = five[five['Chemical'] == 'Heroin']

In [ ]:
len(two.Week.unique())

In [ ]:
tp = pd.concat([two, four, five], axis=0)

In [ ]:
tp.shape

In [ ]:
tempe_fac = pd.read_csv('TEMPEcntMedicalFacilitiesWithin2milesByTract.csv')

In [ ]:
sa = pd.read_csv('sa.csv')

In [ ]:
from shapely.wkt import loads


tempe_fac['geometry'] = tempe_fac['geometry'].apply(loads)

In [ ]:
tempe_fac['geometry'] = gpd.GeoSeries(tempe_fac['geometry'])

In [ ]:
tempe_fac.dtypes

In [ ]:
tempe_fac = gpd.GeoDataFrame(tempe_fac, geometry=tempe_fac.geometry)

In [ ]:
sa['geometry'] = sa['geometry'].apply(loads)

In [ ]:
sa['geometry'] = gpd.GeoSeries(sa['geometry'])

In [ ]:
sa = gpd.GeoDataFrame(sa, geometry=tempe_fac.geometry)

In [ ]:
sa.shape

In [ ]:
tempe_fac.shape

In [ ]:
two.sort_values(by='Sample_Date')

Date Range from 2018-05-14 to 2019-09-15. This is 16 months of data or approx 64 weeks

In [ ]:
tempe_sa = pd.DataFrame()
for i in range(len(sa)):
    df = tempe_fac[tempe_fac.intersects(sa.loc[i,'geometry'])]
    df['site'] = sa.loc[i,'Site']
    tempe_sa = pd.concat([tempe_sa, df], axis=0)

In [ ]:
tempe_sa.head()

In [ ]:
tempe_sa = tempe_sa.groupby('site').agg({'mean'})

In [ ]:
tempe_sa.columns = tempe_sa.columns.droplevel(1)

In [ ]:
tempe_sa

In [ ]:
pnml = tp.merge(tempe_sa, left_on='Site', right_on='site')

In [ ]:
pnml.head()

In [ ]:
pnml.shape

In [ ]:
pnml.columns

In [ ]:
pnml.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
pnml.rename( columns={'Population_Normalized_Mass_Load' : 'pnml' }, inplace=True)

In [ ]:
pnml.replace(-9999.99, np.NaN, inplace = True )
pnml.replace(9999.99, np.NaN, inplace = True )
pnml.replace(-10000, np.NaN, inplace = True )
pnml.replace(10000, np.NaN, inplace = True )

In [ ]:
pnml.describe()

In [ ]:
two = pnml[pnml['Site'] =='TP02']

In [ ]:
len(two.Week.unique())

In [ ]:
four = pnml[pnml['Site'] =='TP04']

In [ ]:
mt = pnml.groupby([pnml['Week'], pnml['Site']]).agg({'mean'})

In [ ]:
mt

In [ ]:
mt.columns = mt.columns.droplevel(1)

In [ ]:
mt.reset_index(inplace=True)

In [ ]:
mt.head()

In [ ]:
mt[mt['Site'] =='TP05']

In [ ]:
plt.plot(two.Week, two.Population_Normalized_Mass_Load)